In [22]:
!pip install tensorflow>=2.12 nltk>=3.8 colorama>=0.4.4 numpy>=1.22 scikit-learn>=1.2 Flask>=2.3



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\yashj\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [34]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yashj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yashj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yashj\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [35]:
import json

intents = {
    "intents": [
        # ... paste the intent data from your example ...
        # (Same as your data above)
    ]
}

with open("intents.json", "w") as f:
    json.dump(intents, f, indent=4)


In [36]:
import json

# Expanded intents with new topics & a fallback
intents_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
            "responses": ["Hello!", "Hi there!", "Greetings! How can I help you?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye"],
            "responses": ["See you later!", "Have a nice day!", "Bye! Come back soon."]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
            "responses": ["Happy to help!", "Any time!", "My pleasure"]
        },
        {
            "tag": "about",
            "patterns": ["Who are you?", "What are you?", "Who you are?"],
            "responses": ["I'm Joana, your personal assistant.", "I’m a chatbot to help you with your questions."]
        },
        {
            "tag": "name",
            "patterns": ["What is your name", "What should I call you", "What's your name?"],
            "responses": ["You can call me Joana.", "I'm Joana!", "Just call me Joana."]
        },
        {
            "tag": "help",
            "patterns": ["Can you help me?", "I need assistance", "I need help", "Support me please"],
            "responses": ["Sure, tell me your problem.", "Yes, please describe the issue."]
        },
        {
            "tag": "google_info",
            "patterns": [
                "What is Google?", "Tell me about Google", "Who is Google?",
                "Explain Google", "Give me info about Google"
            ],
            "responses": [
                "Google is a multinational technology company specializing in internet-related services and products.",
                "Google is best known as a search engine, but it also develops software, hardware, and cloud computing services."
            ]
        },
        {
            "tag": "unknown",
            "patterns": [],
            "responses": ["I'm sorry, I don't understand your question. Could you please rephrase?"]
        }
    ]
}

# Save the updated data
with open("intents.json", "w") as f:
    json.dump(intents_data, f, indent=4)

print("✅ intents.json updated with new topics and fallback intent.")



✅ intents.json updated with new topics and fallback intent.


In [37]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Load intents
with open('intents.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

print("Total patterns:", len(training_sentences))
print("Labels:", labels)

# Encode labels
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels_encoded = lbl_encoder.transform(training_labels)

# Tokenization
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

print("padded_sequences shape:", padded_sequences.shape)


Total patterns: 27
Labels: ['greeting', 'goodbye', 'thanks', 'about', 'name', 'help', 'google_info', 'unknown']
padded_sequences shape: (27, 20)


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(labels), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

epochs = 300
history = model.fit(padded_sequences, np.array(training_labels_encoded), epochs=epochs)


C:\Users\yashj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_3           │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1852 - loss: 2.0778
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.1852 - loss: 2.0762
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.1852 - loss: 2.0746
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.1852 - loss: 2.0731
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.1852 - loss: 2.0715
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.1852 - loss: 2.0699
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.1852 - loss: 2.0682
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.1852 - loss: 2.0665
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.1852 - loss: 2.0649
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.1852 - loss: 2.0631
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.1852 - loss: 2.0613
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.1

In [39]:
import pickle

model.save("chat_model.keras")

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as enc_file:
    pickle.dump(lbl_encoder, enc_file, protocol=pickle.HIGHEST_PROTOCOL)

print("✅ Model and preprocessing objects saved.")



✅ Model and preprocessing objects saved.


In [46]:
def chat():
    # Load trained model
    model = keras.models.load_model('chat_model.keras')

    # Precompile predict function to avoid retracing warnings
    model.make_predict_function()

    # Load tokenizer & label encoder
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    max_len = 20
    confidence_threshold = 0.6

    print(Fore.YELLOW + "Start chatting with the bot (type quit to stop)!" + Style.RESET_ALL)

    # Main chat loop
    while True:
        inp = input(Fore.LIGHTBLUE_EX + "You: " + Style.RESET_ALL)
        if inp.lower() == "quit":
            break

        seq = pad_sequences(tokenizer.texts_to_sequences([inp]),
                            truncating='post', maxlen=max_len)
        result = model.predict(seq)  # ✅ No retracing warning now

        # ... rest of your confidence/fallback/Wikipedia logic ...


In [47]:
import requests, random, re
import colorama
import urllib.parse
from colorama import Fore, Style
from tensorflow import keras

colorama.init()

# Reload intents
with open("intents.json") as file:
    data = json.load(file)

# Wikipedia fetcher
def clean_query(query):
    # Remove common question phrases for better matching
    patterns_to_remove = [
        r'what is ', r'tell me about ', r'who is ',
        r'explain ', r'give me info about ', r'can you explain ',
        r'what are ', r'what\'s ', r'Define ', r'define '
    ]
    query = query.lower()
    for pattern in patterns_to_remove:
        query = re.sub(pattern, '', query)
    return query.strip()

def fetch_wikipedia_summary(user_query):
    try:
        clean_q = clean_query(user_query)
        encoded_query = urllib.parse.quote(clean_q)
        url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{encoded_query}"
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code == 200:
            data = response.json()
            if "extract" in data:
                return data["extract"]
            else:
                print(f"Wikipedia API did not return summary extract for: {clean_q}")
                return None
        else:
            print(f"Wikipedia API returned status code: {response.status_code} for query: {clean_q}")
            return None
    except Exception as e:
        print(f"Error fetching Wikipedia summary for '{user_query}': {e}")
        return None


# Save unknown query for future training
def save_unknown_query(query):
    with open("unknown_queries.txt", "a") as f:
        f.write(query + "\n")

def chat():
    model = keras.models.load_model('chat_model.keras')
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    max_len = 20
    confidence_threshold = 0.6

    print(Fore.YELLOW + "Start chatting with the bot (type quit to stop)!" + Style.RESET_ALL)
    while True:
        inp = input(Fore.LIGHTBLUE_EX + "You: " + Style.RESET_ALL)
        if inp.lower() == "quit":
            break

        seq = pad_sequences(tokenizer.texts_to_sequences([inp]),
                            truncating='post', maxlen=max_len)
        result = model.predict(seq)
        confidence = np.max(result)
        predicted_tag = lbl_encoder.inverse_transform([np.argmax(result)])[0]

        if confidence < confidence_threshold:
            print(Fore.GREEN + "Bot:" + Style.RESET_ALL,
                  "Hmm.. do you want specific detailed information related to your question?")
            save_unknown_query(inp)  # Log unknown
            summary = fetch_wikipedia_summary(inp)
            if summary:
                print(Fore.GREEN + "Bot (Info):" + Style.RESET_ALL, summary)
            else:
                print(Fore.GREEN + "Bot:" + Style.RESET_ALL,
                      "Sorry, I couldn't find more info online.")
        else:
            for intent in data['intents']:
                if intent['tag'] == predicted_tag:
                    print(Fore.GREEN + "Bot:" + Style.RESET_ALL,
                          random.choice(intent['responses']))

chat()


Start chatting with the bot (type quit to stop)!


You:  hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
Bot: Hi there!


You:  what is quantum physics


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Bot: Hmm.. do you want specific detailed information related to your question?
Bot (Info): Quantum mechanics is the fundamental physical theory that describes the behavior of matter and of light; its unusual characteristics typically occur at and below the scale of atoms. It is the foundation of all quantum physics, which includes quantum chemistry, quantum field theory, quantum technology, and quantum information science.


You:  quit
